# New Section

In [ ]:
# buggy_code.py
import tensorflow as tf

# 1. Load Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 2. Preprocess Data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# BUG 1: Labels are not one-hot encoded but loss function expects them to be.

# 3. Build Model
model = tf.keras.Sequential([
    # BUG 2: Input shape is incorrect for a CNN. It's missing the channel dimension.
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # BUG 3: Output layer has wrong number of units and wrong activation for multi-class.
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 4. Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # This loss expects one-hot labels.
              metrics=['accuracy'])

# 5. Train Model
model.fit(x_train, y_train, epochs=5)

In [ ]:
# fixed_code.py
import tensorflow as tf
import numpy as np

# 1. Load Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 2. Preprocess Data
# Reshape input data to add the channel dimension (1 for grayscale)
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# FIX 1: One-hot encode the labels to match the loss function.
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)
# An alternative fix would be to keep integer labels and change loss to 'sparse_categorical_crossentropy'.

# 3. Build Model
model = tf.keras.Sequential([
    # FIX 2: Corrected input_shape to (28, 28, 1).
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)), # Added for better performance
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # FIX 3: Output layer must have 10 units (for 10 digits) and 'softmax' activation
    # for multi-class probability distribution.
    tf.keras.layers.Dense(10, activation='softmax')
])

# 4. Compile Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # This loss now matches the one-hot labels.
              metrics=['accuracy'])

model.summary()

# 5. Train Model
print("\nTraining fixed model...")
model.fit(x_train, y_train_cat, epochs=5, validation_data=(x_test, y_test_cat))
print("Training complete.")